In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [105]:
white_wine_new = pd.read_csv('C:/Users/raclo/Documents/Bootcamp/Project 2/Sprint 2/white_wine_modified.csv')
white_wine_new.head()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_label,alcohol_level,ph_level,wine_type
0,0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,medium,low,low,white
1,1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,medium,low,regular,white
2,2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,medium,regular,regular,white
3,3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,medium,low,regular,white
4,6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6,medium,low,regular,white


In [107]:
white_wine_new.describe()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000,3961.000000
mean,2414.467054,6.839346,0.280538,0.334332,5.914819,0.045905,34.889169,137.193512,0.993790,3.195458,0.490351,10.589358,5.854835
std,1426.025020,0.866860,0.103437,0.122446,4.861646,0.023103,17.210021,43.129065,0.002905,0.151546,0.113523,1.217076,0.890683
min,0.000000,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,1170.000000,6.300000,0.210000,0.270000,1.600000,0.035000,23.000000,106.000000,0.991620,3.090000,0.410000,9.500000,5.000000
50%,2385.000000,6.800000,0.260000,0.320000,4.700000,0.042000,33.000000,133.000000,0.993500,3.180000,0.480000,10.400000,6.000000
75%,3641.000000,7.300000,0.330000,0.390000,8.900000,0.050000,45.000000,166.000000,0.995710,3.290000,0.550000,11.400000,6.000000
max,4897.000000,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [92]:
# Identify duplicate rows
duplicate_rows = white_wine_new.duplicated()

# Drop the duplicates
white_wine_new.drop_duplicates(inplace=True)

# Count the duplicates
duplicate_rows.value_counts()

False    3961
Name: count, dtype: int64

In [93]:
white_wine_new.drop(columns=["Unnamed: 0", "alcohol_level","ph_level","wine_type", "quality"], inplace=True)

In [94]:
white_wine_new[['quality_label']].head()

,quality_label
0,medium
1,medium
2,medium
3,medium
4,medium


In [95]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

oe = OrdinalEncoder(categories=[["low", "medium", "high"]], dtype=int) # this is how you can define the label order
white_wine_new['quality_label'] = oe.fit_transform(white_wine_new[['quality_label']])

white_wine_new[['quality_label']].head()

,quality_label
0,1
1,1
2,1
3,1
4,1


In [108]:
white_wine_new.drop(columns=["free sulfur dioxide"], inplace=True)

In [109]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Define features and target variable
X = white_wine_new.drop('quality_label', axis=1)
y = white_wine_new['quality_label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)


In [98]:
# Apply SMOTE only on the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)


In [99]:
# Apply SMOTE on the test data (for experimentation)
X_test_smote, y_test_smote = smote.fit_resample(X_test, y_test)


In [110]:
# Normalize the data using StandardScaler
scaler = StandardScaler()

# Fit on training data and transform both train and test sets
X_train_smote_norm = scaler.fit_transform(X_train_smote)
X_test_smote_norm = scaler.transform(X_test_smote)  # Applying SMOTE-transformed test set


In [111]:
# Define and train the model
RF_clf = RandomForestClassifier(random_state=42)
RF_clf.fit(X_train_smote_norm, y_train_smote)

# Make predictions on the SMOTE-enhanced test set
RF_preds = RF_clf.predict(X_test_smote_norm)


In [112]:
# Compute accuracy on the SMOTE-augmented test set
accuracy = accuracy_score(y_test_smote, RF_preds)
print(f"Accuracy on SMOTE-enhanced test set: {accuracy:.4f}")

# Print classification report
print(classification_report(y_test_smote, RF_preds))

# Print confusion matrix
print(confusion_matrix(y_test_smote, RF_preds))


Accuracy on SMOTE-enhanced test set: 0.5645
              precision    recall  f1-score   support

           0       0.75      0.68      0.71       496
           1       0.41      0.70      0.52       496
           2       0.80      0.31      0.44       496

    accuracy                           0.56      1488
   macro avg       0.65      0.56      0.56      1488
weighted avg       0.65      0.56      0.56      1488

[[339 156   1]
 [109 349  38]
 [  5 339 152]]
